In [2]:
import pandas as pd
import requests
from datetime import date, timedelta
import plotly.graph_objects as go
import plotly.express as px
import json
from pymongo import MongoClient 


In [3]:
client = MongoClient() 
client = MongoClient("mongodb://localhost:27017")

In [4]:
mydatabase = client['Zones'] 
mycollection = mydatabase['Nifty50D'] 

In [5]:
def fiblvlprice(lvl,ind):
    return (explosive['Low'][ind]-explosive['High'][ind])*lvl+explosive['High'][ind]


In [6]:
def wwma(values, n):
    """
     J. Welles Wilder's EMA 
    """
    return values.ewm(alpha=1/n, adjust=False).mean()

def atr(df, n=14):
    data = df.copy()
    high = data['high']
    low = data['low']
    close = data['close']
    data['tr0'] = abs(high - low)
    data['tr1'] = abs(high - close.shift())
    data['tr2'] = abs(low - close.shift())
    tr = data[['tr0', 'tr1', 'tr2']].max(axis=1)
    atr = wwma(tr, n)
    return atr

In [7]:
def isCandleExplosive(high,low,openn,close,atravg):
    mp = 1.2
    ratio = 50
    cand = 20
    rang = atravg

    candr = abs(high-low)
    bodyr = abs(openn-close)
    if (bodyr/candr)>=ratio/100 and candr>=(rang*mp):
        return True
    else:
        return False

def get_mondays_fridays(a, b):

    start_date = pd.to_datetime(a)
    end_date = pd.to_datetime(b)

  # Ensure start_date is earlier or equal to end_date
    if start_date > end_date:
        start_date, end_date = end_date, start_date

  # Create a date range (optimized for efficiency)
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')

  # Filter for Mondays and Fridays using boolean indexing
    mondays = date_range[date_range.weekday == 0]
    fridays = date_range[date_range.weekday == 4]

    return mondays, fridays

In [8]:
def checkdata(df):
    ind = []
    date = []
    o = []
    h = []
    l = []
    c = []
    explosive = pd.DataFrame()

    for i in range(0,len(df['date'])-1):
        if isCandleExplosive(df['high'][i],df['low'][i],df['open'][i],df['close'][i],df['atravg'][i]):
            #print(df['date'][i],df['atr'][i])
            ind.append(i)
            date.append(df['date'][i])
            o.append(df['open'][i])
            h.append(df['high'][i])
            l.append(df['low'][i])
            c.append(df['close'][i])
    explosive['index'] = ind
    explosive['Date'] = date
    explosive['Open'] = o
    explosive['High'] = h
    explosive['Low'] = l
    explosive['Close'] = c
    return explosive

In [101]:

def fetchData(id,frm,to):
    url = "https://kite.zerodha.com/oms/instruments/historical/"+id+"/day?user_id=EM9701&oi=1&from="+frm+"&to="+to+""
    #url = "https://kite.zerodha.com/oms/instruments/historical/"+id+"/60minute?user_id=EM9701&oi=1&from=2023-11-21&to=2024-02-19"
    headers = {
        "accept": "*/*",
        "accept-language": "en-IN,en-GB;q=0.9,en-US;q=0.8,en;q=0.7",
        "authorization": "enctoken csVmgO1+aNwREY03J187dXCtenQ3SzydfN6T/GFZtxx8uMUD6WmrEy9hIgvlKiV1KKQNjDohBwG4HW1yhrehOqjEaaRiv1RrTQQwPYVb5CFm9au/pMooZA==",  # Replace with your actual authorization token
        "sec-ch-ua": "\"Not A(Brand\";v=\"99\", \"Google Chrome\";v=\"121\", \"Chromium\";v=\"121\"",
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": "\"Windows\"",
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "same-origin"
    }

    response = requests.get(url, headers=headers)

    # Check for successful response
    if response.status_code == 200:
        #data = response.json()  # Assuming response is JSON
        # Process the data here
        #print(data)
        return response.json()
    else:
        print(f"Error: {response.status_code}")

    

In [102]:
def createDffromData(data):
    column_names = ['date', 'open', 'high', 'low', 'close', 'volume', 'oi']
    df = pd.DataFrame(data['data']['candles'], columns=column_names)
    for i in range(0,len(df['open'])):
        df['date'][i]=df['date'][i].split('T')[0]
    return df

In [12]:
def createDffromDataS(data):
    column_names = ['date', 'open', 'high', 'low', 'close', 'volume', 'oi']
    df = pd.DataFrame(data['data']['candles'], columns=column_names)
    return df

In [13]:
def putAtrValues(df):
    atri = atr(df,1)
    atr_avg = atr(df,20)
    df = df.merge(atr_avg.rename('atravg'), left_index=True, right_index=True)
    df = df.merge(atri.rename('atr'),left_index=True, right_index=True)
    return df

In [14]:
def saveDataframes(collection):
    for i in range(0,len(tiks)):
        a = createDffromDataS(data[i][tiks[i]])
        c = a.to_json()
        print(collection.insert_one({'ticker':b[i],'dataframe':str(c)}))
    

In [15]:
def jsonToDf(ticker,collection):
    return pd.read_json((collection.find_one({'ticker':ticker}))['dataframe'])

In [16]:

def greenExplosive(explosive,df):
#Green explosive
    obj = []
    for i in range(0,len(explosive['Date'])-1):
        base = explosive['index'][i]-1
        if explosive['Close'][i] > explosive['Open'][i] and abs(df['close'][base]-df['open'][base]) <= (explosive['High'][i]-explosive['Low'][i])*0.2 and explosive['Close'][i] > fiblvlprice(0.4,i):
            obj.append({"l":1,"legoutDate":explosive['Date'][i],"baseHigh":df['high'][base],"baseLow":df['low'][base],"dist":((df['high'][base]-ltp)/ltp)*100})
            #epm and open[1] > open and close[1] < low and open[1] < fibprice(0.75) and high-low > high[1]-low[1]
        elif explosive['Close'][i] > explosive['Open'][i] and df['open'][base] > explosive['Open'][i] and df['close'][base] < explosive['Low'][i] and df['open'][base] < fiblvlprice(0.75,i) and explosive['High'][i]-explosive['Low'][i] > df['high'][base]-df['low'][base]:
            obj.append({"l":2,"legoutDate":explosive['Date'][i],"baseHigh":df['high'][base],"baseLow":df['low'][base],"dist":((df['high'][base]-ltp)/ltp)*100})
            #close > open and epm and close[1] < fibprice(0.95) and open[1] > fibprice(1.25) and close[1] > open[1] and close > fibprice(0.2)
        elif explosive['Close'][i] > explosive['Open'][i] and df['close'][base] < fiblvlprice(0.95,i) and df['open'][base] > fiblvlprice(1.25,i) and df['close'][base] > df['open'][base] and explosive['Close'][i] > fiblvlprice(0.4,i):
            obj.append({"l":3,"legoutDate":explosive['Date'][i],"baseHigh":df['high'][base],"baseLow":df['low'][base],"dist":((df['high'][base]-ltp)/ltp)*100})
            #epm and checkfullbar(0) and checkfullbar(1) and high[2]-low[2] < high[1]-low[1] and checkbarexp(1) and close > open and open[2] < fibpriceind(0.5,1) and close[2] < fibpriceind(0.5,1)
    return obj

In [17]:
def find_nearest_dist_to_zero(data):
    results = []
    for token_item in data:
        nearest_dist = float('inf')
        nearest_zone = None

        for zone_item in token_item['zone']:
            dist = zone_item['dist']

            # Skip non-negative 'dist' values to focus on negative ones
            if dist >= 0:
                continue

            # Calculate absolute difference for nearest to zero comparison
            abs_diff = abs(dist)

            # Update nearest_dist and nearest_zone if closer to zero
            if abs_diff < nearest_dist:
                nearest_dist = abs_diff
                nearest_zone = zone_item

        results.append({
            'token': token_item['token'],
            'nearest_dist': nearest_dist,
            'nearest_zone': nearest_zone
        })

    return results


# Main Code

In [166]:
#tiks = [6401,3861249,40193,60417,1510401,4267265,4268801,81153,2714625,134657,140033,177665,5215745,2800641,225537,232961,315393,1850625,341249,119553,345089]
#tiks = [1793,3529217,3394561,3930881,951809,4278529,486657,7712001,3484417,2955009,3660545,5728513,2455041,2763265]
#tiks = [738561]
data = []
for i in range(0,len(tiks)):
    data.append({tiks[i]:fetchData(str(tiks[i]),'2023-02-14','2024-02-19')})
#data = fetchData('160769','2023-02-14','2024-02-14')


In [167]:
dmzs = []
for j in range(0,len(tiks)):
    df = createDffromData(data[j][tiks[j]])
    ltp = df['close'].iloc[-1]
    print(ltp)
    print(tiks[j])
    df = putAtrValues(df)
    explosive1 = checkdata(df)
    explosive = explosive1.iloc[::-1].reset_index(drop=True)
    #print(explosive)
    obj = []
    for i in range(0,len(explosive['Date'])-1):
        base = explosive['index'][i]-1
        if explosive['Close'][i] > explosive['Open'][i] and abs(df['close'][base]-df['open'][base]) <= (explosive['High'][i]-explosive['Low'][i])*0.2 and explosive['Close'][i] > fiblvlprice(0.7,i) and df['open'][base] < fiblvlprice(0.5,i) and df['close'][base] < fiblvlprice(0.5,i) and explosive['Open'][i] < fiblvlprice(0.3,i) and df['high'][base] < fiblvlprice(0.5,i):
            obj.append(
                {
                    "l":1,
                    "legoutDate":explosive['Date'][i],
                    "baseHigh":df['high'][base],
                    "baseLow":df['low'][base],
                    "dist":((df['high'][base]-ltp)/ltp)*100,
                    "ltp":ltp,
                    "chartData":{
                        "series":[
                            {
                                "data":
                                [
                                        {
                                "x":df['date'][base],
                                "y":[df['open'][base],df['high'][base],df['low'][base],df['close'][base]]
                            },
                            {
                                "x":explosive['Date'][i],
                                "y":[explosive['Open'][i],explosive['High'][i],explosive['Low'][i],explosive['Close'][i]]
                            }
                                ]
                            }
                        ],
                        "legoutDate":explosive['Date'][i]
                    }
                })
            #epm and open[1] > open and close[1] < low and open[1] < fibprice(0.75) and high-low > high[1]-low[1]
        elif explosive['Close'][i] > explosive['Open'][i] and df['open'][base] > explosive['Open'][i] and df['close'][base] < explosive['Low'][i] and df['open'][base] < fiblvlprice(0.75,i) and explosive['High'][i]-explosive['Low'][i] > df['high'][base]-df['low'][base] and (explosive['Close'][i]-explosive['Close'][i])*0.25 > (df['open'][base]-df['close'][base]):
            obj.append(
                {
                    "l":2,
                    "legoutDate":explosive['Date'][i],
                    "baseHigh":df['high'][base],
                    "baseLow":df['low'][base],
                    "dist":((df['high'][base]-ltp)/ltp)*100,
                    "ltp":ltp,
                    "chartData":{
                        "series":[
                            {
                                "data":
                                [
                                        {
                                "x":df['date'][base],
                                "y":[df['open'][base],df['high'][base],df['low'][base],df['close'][base]]
                            },
                            {
                                "x":explosive['Date'][i],
                                "y":[explosive['Open'][i],explosive['High'][i],explosive['Low'][i],explosive['Close'][i]]
                            }
                                ]
                            }
                        ],
                        "legoutDate":explosive['Date'][i]
                    }
                })
            #close > open and epm and close[1] < fibprice(0.95) and open[1] > fibprice(1.25) and close[1] > open[1] and close > fibprice(0.2)
        elif explosive['Close'][i] > explosive['Open'][i] and df['close'][base] < fiblvlprice(0.95,i) and df['open'][base] > fiblvlprice(1.25,i) and df['close'][base] > df['open'][base] and explosive['Close'][i] > fiblvlprice(0.4,i):
            obj.append(
                {
                    "l":3,
                    "legoutDate":explosive['Date'][i],
                    "baseHigh":df['high'][base],
                    "baseLow":df['low'][base],
                    "dist":((df['high'][base]-ltp)/ltp)*100,
                    "ltp":ltp,
                    "chartData":{
                        "series":[
                            {
                                "data":
                                [
                                        {
                                "x":df['date'][base],
                                "y":[df['open'][base],df['high'][base],df['low'][base],df['close'][base]]
                            },
                            {
                                "x":explosive['Date'][i],
                                "y":[explosive['Open'][i],explosive['High'][i],explosive['Low'][i],explosive['Close'][i]]
                            }
                                ]
                            }
                        ],
                        "legoutDate":explosive['Date'][i]
                    }
                })
            #close > open and epm and close[1] < fibprice(0.95) and open[1] > fibprice(1.25) and close[1] > open[1] and close > fibprice(0.2)
        elif explosive['Close'][i] > explosive['Open'][i] and df['close'][base] < fiblvlprice(0.95,i) and df['open'][base] > fiblvlprice(1.3,i) and df['close'][base] > df['open'][base] and explosive['Close'][i] > fiblvlprice(0.2,i):
            obj.append(
                {
                    "l":4,
                    "legoutDate":explosive['Date'][i],
                    "baseHigh":df['high'][base],
                    "baseLow":df['low'][base],
                    "dist":((df['high'][base]-ltp)/ltp)*100,
                    "ltp":ltp,
                    "chartData":{
                        "series":[
                            {
                                "data":
                                [
                                        {
                                "x":df['date'][base],
                                "y":[df['open'][base],df['high'][base],df['low'][base],df['close'][base]]
                            },
                            {
                                "x":explosive['Date'][i],
                                "y":[explosive['Open'][i],explosive['High'][i],explosive['Low'][i],explosive['Close'][i]]
                            }
                                ]
                            }
                        ],
                        "legoutDate":explosive['Date'][i]
                    }
                })
            
    dmzs.append({tiks[j]:obj})

C:\Users\Admin\AppData\Local\Temp\ipykernel_276\1498270930.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'][i]=df['date'][i].split('T')[0]
C:\Users\Admin\AppData\Local\Temp\ipykernel_276\3380140627.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if (bodyr/candr)>=ratio/100 and candr>=(rang*mp):
C:\Users\Admin\AppData\Local\Temp\ipykernel_276\1498270930.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'][i]=df['date'][i].split('T')[0]
C:\Users\Admin\AppData\Local\Temp\ipykernel_276\3380140627.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if (bodyr/candr)

1011.7
1552897
366.45
2076161
8876.25
78081
270.7
1195009
188.75
98049


C:\Users\Admin\AppData\Local\Temp\ipykernel_276\1498270930.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'][i]=df['date'][i].split('T')[0]


# Sorts Zone

In [168]:
zones = []
#b=['TCS']
for ti in range(0,len(data)):
    brokenzone = []
    df = createDffromData(data[ti][tiks[ti]])
    for i in range (0,len(dmzs[ti][tiks[ti]])):
        for j in range(0,len(df)):
            
            if df['date'][j] == dmzs[ti][tiks[ti]][i]['chartData']['legoutDate']:
                #print(df['date'][j])
                for k in range(j+1,len(df)):
                    #print(k,df['date'][k],df['low'][k],dmzs[0][tiks[0]][i]['baseLow'])
                    if dmzs[ti][tiks[ti]][i]['baseLow'] > df['low'][k]:
                        #f0e0rti,dmzs[ti][tiks[ti]][i]['legoutDate'],'broken',dmzs[ti][tiks[ti]][i]['baseLow'],df['date'][k])
                        brokenzone.append(dmzs[ti][tiks[ti]][i]['chartData']['legoutDate'])
                        break
    a = dmzs[ti][tiks[ti]]
    bz_set = set(brokenzone)
    #print(tiks[ti])
    # Efficiently remove matching dictionaries from the end
    for i in range(len(a) - 1, -1, -1):
        if a[i]['legoutDate'] in bz_set:
            del a[i]
    #print(json.dumps(a,indent=4))
    zones.append({'ticker':b[ti],'token':tiks[ti],'zone':a})

C:\Users\Admin\AppData\Local\Temp\ipykernel_276\1498270930.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'][i]=df['date'][i].split('T')[0]
C:\Users\Admin\AppData\Local\Temp\ipykernel_276\1498270930.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'][i]=df['date'][i].split('T')[0]
C:\Users\Admin\AppData\Local\Temp\ipykernel_276\1498270930.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'][i]=df['date

In [169]:
a = find_nearest_dist_to_zero(zones)

# Print Nearest Zones

In [170]:
for i in range(0,len(a)):
    if a[i]['nearest_zone'] != None:
        print(i,'token: ',b[i],'legoutDate: ',a[i]['nearest_zone']['legoutDate'],a[i]['nearest_zone']['dist'],a[i]['nearest_zone']['baseHigh'])

0 token:  ATGL legoutDate:  2023-11-28 -46.239003657210645 543.9
1 token:  AWL legoutDate:  2023-11-28 -12.40278346295538 321.0
2 token:  BAJAJHLDNG legoutDate:  2024-01-03 -11.573581185748488 7848.95
3 token:  BANKBARODA legoutDate:  2023-12-13 -19.782046545991868 217.15
4 token:  BEL legoutDate:  2023-12-15 -12.927152317880797 164.35


In [171]:
len(tiks)

5

In [172]:
#tik = 1
for i in range(0,len(zones)):
    print(mycollection.insert_one(zones[i]))

In [119]:
zones[1]

{'ticker': 'PIDILITIND',
 'token': '681985',
 'zone': [{'l': 1,
   'legoutDate': '2023-11-30',
   'baseHigh': 2515.9,
   'baseLow': 2485.0,
   'dist': -7.20174095863379,
   'ltp': 2711.15,
   'chartData': {'series': [{'data': [{'x': '2023-11-29',
        'y': [2488.95, 2515.9, 2485.0, 2503.6]},
       {'x': '2023-11-30', 'y': [2503.6, 2563.6, 2487.0, 2552.35]}]}],
    'legoutDate': '2023-11-30'}},
  {'l': 1,
   'legoutDate': '2023-11-17',
   'baseHigh': 2466.15,
   'baseLow': 2446.05,
   'dist': -9.036755620308725,
   'ltp': 2711.15,
   'chartData': {'series': [{'data': [{'x': '2023-11-16',
        'y': [2460.5, 2466.15, 2446.05, 2459.1]},
       {'x': '2023-11-17', 'y': [2468.85, 2534.3, 2465.05, 2510.3]}]}],
    'legoutDate': '2023-11-17'}},
  {'l': 1,
   'legoutDate': '2023-10-31',
   'baseHigh': 2330.0,
   'baseLow': 2299.0,
   'dist': -14.058609815023148,
   'ltp': 2711.15,
   'chartData': {'series': [{'data': [{'x': '2023-10-30',
        'y': [2330.0, 2330.0, 2299.0, 2312.15]},
 

In [16]:
zones[0]

NameError: name 'zones' is not defined

In [132]:
def convert_to_dict(data):
    result = {}
    for item in data:
        key, value = item.split("/")
        result[key] = value
    return result

# Example usage
data =["ABB/3329", "ACC/5633", "ADANIGREEN/912129", "AARTIIND/1793", "AMBUJACEM/325121",
       "ATGL/1552897", "AWL/2076161", "BAJAJHLDNG/78081", "BANKBARODA/1195009", "BEL/98049",
       "BERGEPAINT/103425", "CANBK/2763265", "CHOLAFIN/175361", "COLPAL/3876097", "DABUR/197633",
       "DLF/3771393", "DMART/5097729", "GAIL/1207553", "GODREJCP/2585345", "HAL/589569",
       "HAVELLS/2513665", "HDFCAMC/1086465", "ICICIGI/5573121", "ICICIPRULI/4774913", "INDIGO/2865921",
       "INDUSTOWER/7458561", "IOC/415745", "IRCTC/3484417", "JINDALSTEL/1723649", "LICI/2426881",
       "MARICO/1041153", "MCDOWELL-N/2674433", "MOTHERSON/1076225", "MUTHOOTFIN/6054401", "NAUKRI/3520257",
       "NYKAA/1675521", "PIDILITIND/681985", "PIIND/6191105", "SBICARD/4600577", "SIEMENS/806401",
       "SRF/837889", "TATAPOWER/877057", "TORNTPHARM/900609", "VBL/4843777", "VEDL/784129", "ZOMATO/1304833"]

# your full data here
dictionary = convert_to_dict(data)

a = {"NiftyNxt50":dictionary}


In [133]:
token =a #['NiftyNxt50']['ABB']

In [134]:
a

{'NiftyNxt50': {'ABB': '3329',
  'ACC': '5633',
  'ADANIGREEN': '912129',
  'AARTIIND': '1793',
  'AMBUJACEM': '325121',
  'ATGL': '1552897',
  'AWL': '2076161',
  'BAJAJHLDNG': '78081',
  'BANKBARODA': '1195009',
  'BEL': '98049',
  'BERGEPAINT': '103425',
  'CANBK': '2763265',
  'CHOLAFIN': '175361',
  'COLPAL': '3876097',
  'DABUR': '197633',
  'DLF': '3771393',
  'DMART': '5097729',
  'GAIL': '1207553',
  'GODREJCP': '2585345',
  'HAL': '589569',
  'HAVELLS': '2513665',
  'HDFCAMC': '1086465',
  'ICICIGI': '5573121',
  'ICICIPRULI': '4774913',
  'INDIGO': '2865921',
  'INDUSTOWER': '7458561',
  'IOC': '415745',
  'IRCTC': '3484417',
  'JINDALSTEL': '1723649',
  'LICI': '2426881',
  'MARICO': '1041153',
  'MCDOWELL-N': '2674433',
  'MOTHERSON': '1076225',
  'MUTHOOTFIN': '6054401',
  'NAUKRI': '3520257',
  'NYKAA': '1675521',
  'PIDILITIND': '681985',
  'PIIND': '6191105',
  'SBICARD': '4600577',
  'SIEMENS': '806401',
  'SRF': '837889',
  'TATAPOWER': '877057',
  'TORNTPHARM': '900

In [135]:
keys = []
values = []

for key, value in a['NiftyNxt50'].items():
    keys.append(key)
    values.append(value)

print("Keys:", keys)
print("Values:", values)

Keys: ['ABB', 'ACC', 'ADANIGREEN', 'AARTIIND', 'AMBUJACEM', 'ATGL', 'AWL', 'BAJAJHLDNG', 'BANKBARODA', 'BEL', 'BERGEPAINT', 'CANBK', 'CHOLAFIN', 'COLPAL', 'DABUR', 'DLF', 'DMART', 'GAIL', 'GODREJCP', 'HAL', 'HAVELLS', 'HDFCAMC', 'ICICIGI', 'ICICIPRULI', 'INDIGO', 'INDUSTOWER', 'IOC', 'IRCTC', 'JINDALSTEL', 'LICI', 'MARICO', 'MCDOWELL-N', 'MOTHERSON', 'MUTHOOTFIN', 'NAUKRI', 'NYKAA', 'PIDILITIND', 'PIIND', 'SBICARD', 'SIEMENS', 'SRF', 'TATAPOWER', 'TORNTPHARM', 'VBL', 'VEDL', 'ZOMATO']
Values: ['3329', '5633', '912129', '1793', '325121', '1552897', '2076161', '78081', '1195009', '98049', '103425', '2763265', '175361', '3876097', '197633', '3771393', '5097729', '1207553', '2585345', '589569', '2513665', '1086465', '5573121', '4774913', '2865921', '7458561', '415745', '3484417', '1723649', '2426881', '1041153', '2674433', '1076225', '6054401', '3520257', '1675521', '681985', '6191105', '4600577', '806401', '837889', '877057', '900609', '4843777', '784129', '1304833']


In [163]:
tiks = []
b = []
for i in range(5,10):
    b.append(keys[i])
    tiks.append(values[i])

In [164]:
tiks

['1552897', '2076161', '78081', '1195009', '98049']

In [165]:
b

['ATGL', 'AWL', 'BAJAJHLDNG', 'BANKBARODA', 'BEL']